## Reading, exploring data

In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
import urllib.request

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
filename = 'input.txt'

# download file
urllib.request.urlretrieve(url, filename)

('input.txt', <http.client.HTTPMessage at 0x1addcd25df0>)

In [3]:
# read it in to inspect it
with open(filename, 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print('length of dataset in characters : ', len(text))

length of dataset in characters :  1115394


In [5]:
# let's look at the first 1000 charaacters
print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('vocab size :', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size : 65


## Tokenization, train / val split

In [7]:
#create a mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder : take a list of intergers, output a string

print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# let's now encode the entire text dataset and store it into a torch.tensor
import torch # we use PyTorch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100]) # the 1000 characters we looked at earlier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# Let's now split up the data into trainn and validation sets
n = int(0.9 * len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:] 

In [10]:
# chunk = block
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size] # inputs to the transformer
y = train_data[1:block_size+1]  
for t in range(block_size):
    context = x[:t+1]   
    target = y[t]
    print(f'when input is {context} the target : ', {target})

when input is tensor([18]) the target :  {tensor(47)}
when input is tensor([18, 47]) the target :  {tensor(56)}
when input is tensor([18, 47, 56]) the target :  {tensor(57)}
when input is tensor([18, 47, 56, 57]) the target :  {tensor(58)}
when input is tensor([18, 47, 56, 57, 58]) the target :  {tensor(1)}
when input is tensor([18, 47, 56, 57, 58,  1]) the target :  {tensor(15)}
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target :  {tensor(47)}
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target :  {tensor(58)}


In [12]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    # data는 여기서 불러온다.
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # (high, size)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    # return (batch_size) x (block_size) tensor = 4 * 8 
    return x, y

xb, yb = get_batch('train')
print('input:')
print(xb.shape)
print(xb)
print('target:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} the target : {target}')


input:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target : 43
when input is [24, 43] the target : 58
when input is [24, 43, 58] the target : 5
when input is [24, 43, 58, 5] the target : 57
when input is [24, 43, 58, 5, 57] the target : 1
when input is [24, 43, 58, 5, 57, 1] the target : 46
when input is [24, 43, 58, 5, 57, 1, 46] the target : 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target : 39
when input is [44] the target : 53
when input is [44, 53] the target : 56
when input is [44, 53, 56] the target : 1
when input is [44, 53, 56, 1] the target : 58
when input is [44, 53, 56, 1, 58] the target : 46
when input

In [13]:
print(xb) # our input to the transformer 

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


### simplest baseline : bigram language model, loss, geeration

In [14]:
import torch
import torch.nn as nn 
from torch.nn import functional as F

torch.manual_seed(1337)  

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C) 
        # B : batch size
        # T : Time -> block size
        # C : channel, at this situations, vocab_size 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C) # 2 dimensional
            targets = targets.view(B*T) # 1 dimensional
            loss = F.cross_entropy(logits, targets) # negative log likelihood
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            # self(idx)는 뭘까?
            # nn.Module을 상속받았기에, 자동으로 __call__ 메서드를 갖으며,
            # forward 메서드로 연결된다.
            # 즉, self(idx) == self.forward(idx)
            logits, loss = self(idx)
            # focus only on the last time step, 
            # we pluck out last element at the time dimension
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilities (vocab들의 분포 확률을 구한다.)
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution (distribution에서 뽑는다.)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence (뽑은 결과를 idx에 새로 추가한다.)
            idx = torch.cat((idx,idx_next), dim=1)  # (B, T+1)
        return idx
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb,yb)
print(logits.shape)    
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(idx = torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [15]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

In [19]:
batch_size = 32
for steps in range(1000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5059468746185303


In [23]:
print(decode(model.generate(idx = torch.zeros((1,1),dtype=torch.long),max_new_tokens=500)[0].tolist()))


JqIIrdithomfas
ARDUPLEVw nkkOMyiy p'd?w teno t me,,
VIsucel INCENA-thathe zy n the ss TInyNue'mma!
Qbbme nMm!
Wh,

NA3Scoulgdant
If wiqun ar thie d
mg t,
Q
NAfaup, &MNI3QKINu'd cear3Tbore c'lasseASA d., rthifr den,
l wp mavexzqum HA muesu, al, fainkn batt.

L$3licqf, slou d Elotim:CK&y;whalfau-, eDWghzI maZGSin.
Gluand mee.
pNGeppbrIOKANDYONAR:Miven!UENRlli-relvoubt nsomy; orh men;
K: is s wino, thedumy seldeanuly, chacPJut f ds ppx;
SCEYou cexu my,m.
NduXrdud fCI:Kodhs whete
Sinorq-thO n g!othi


### The mathematical trick in self-attentionb

In [2]:
# consider the following toy example:
import torch
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [4]:
# version 1
# We want x[b, t] = mean_{i <=t} x[b,i]
# 위 뜻은, 정보 전달이(여기선 단순히 평균으로 전달) 현재 토큰 기준으로, 오직 이전 순서의 토큰과 이루어져야한다는 것이다.

xbow =  torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev, dim = 0)

In [18]:
# version 2
# more efficient way : using matrix multiply as weighted aggregation
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x # (1,T,T) @ (B,T,C) ---> (B,T,C)

# broad casting의 원칙 1 : 비어있는 차원을 1로 추가한다.
# ex) (T,T) -> (1,T,T)
# broad casting의 원칙 2 : 두 차원의 연산중에 있어서, 적어도 차원 하나는 값이 같아야 한다.

In [19]:
torch.allclose(xbow, xbow2)
# false가 나온다면 부동소숫점 연산 때문일듯.

False

In [26]:
wei = torch.zeros((T,T))
wei = wei.masked_fill(mask = tril == 0, value = float('-inf')) # Fills elements of self tensor with `value` where `mask` is True
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [34]:
# version 3 : use Softmax
from torch.nn import functional as F
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
 # `dim = -1` means last dimension, i.e., dimension 1 for a (T,T) tensor.

In [35]:
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

$Softmax(x_i) =  \frac{exp(x_i)}{\sum_j{exp(x_j)}}$ 이므로,
- '0' -> 1
- '-inf'

In [ ]:
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x

#### matrix multiply as weighted aggregation
행렬 곱셈으로 평균을 구하는 trick

In [9]:
torch.tril(torch.ones(3,3)) # .tril : Returns the lower triangular part of the matrix

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [11]:


torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, dim = 1, keepdim = True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a = ')
print(a)
print('--')
print('b = ')
print(b)
print('--')
print('c = ')
print(c)

a = 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b = 
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c = 
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
